<a href="https://colab.research.google.com/github/Orient12/HelloWorld/blob/master/transformer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>欢迎使用 Colaboratory！</h1>


Colaboratory 是一个免费的 Jupyter 笔记本环境，不需要进行任何设置就可以使用，并且完全在云端运行。

借助 Colaboratory，您可以编写和执行代码、保存和共享分析结果，以及利用强大的计算资源，所有这些都可通过浏览器免费使用。

In [23]:
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
!pip install tqdm
!pip install pandas
!pip install numpy
!pip install nltk


In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import json
import torch.utils.data.dataset as Dataset
import torch.optim as optim

In [0]:
class constants():
  def __init__(self):
    self.PAD = 0
    self.UNK = 1
    self.BOS = 2
    self.EOS = 3

    self.PAD_WORD = '<blank>'
    self.UNK_WORD = '<unk>'
    self.BOS_WORD = '<s>'
    self.EOS_WORD = '</s>'
Constants = constants()

In [0]:
class MultiHeadedAttention(torch.nn.Module):
    def __init__(self, model_dim=512, headed_count=8, dropout=0.0):
        super(MultiHeadedAttention, self).__init__()
        self.headed_count = headed_count
        self.head_dim = model_dim//headed_count
        self.model_dim = model_dim
        self.linear_q = torch.nn.Linear(model_dim, headed_count * self.head_dim)
        self.linear_k = torch.nn.Linear(model_dim, headed_count * self.head_dim)
        self.linear_v = torch.nn.Linear(model_dim, headed_count * self.head_dim)
        self.linear_final = torch.nn.Linear(model_dim, model_dim)
        self.ScaledDotProductAttention = ScaledDotProductAttention(dropout)
        self.dropout = torch.nn.Dropout(dropout)
        self.layer_norm = torch.nn.LayerNorm(model_dim)
    def forward(self, query, key, value, Mask=None):

        """
        :param X: 输入张量， （Batch_size, Sentence_Length, model_dim）
        :param Mask: Mask张量， (Sentence_Length, Sentence_Length)
        :return: output, (batch_size, Sentence_Length, model_dim)
        """
        X_size = query.size()
        batch_size = X_size[0]
        sentence_length = X_size[1]
        model_dim = X_size[2]

        residual = query

        #生成query,key,value向量
        query = self.linear_q(query)
        key = self.linear_k(key)
        value = self.linear_v(value)

        #分出headed_count个头
        query = query.view(batch_size * self.headed_count, -1, self.head_dim)
        key = key.view(batch_size * self.headed_count, -1, self.head_dim)
        value = value.view(batch_size * self.headed_count, -1, self.head_dim)

        # if Mask:
        Mask = Mask.repeat(self.headed_count, 1, 1)

        scale = self.head_dim ** -0.5

        #ScaledDotAttention
        context, attention = self.ScaledDotProductAttention(query, key, value, scale=scale, Mask=Mask)

        #concat head
        context = context.view(batch_size, -1, self.headed_count * self.head_dim)

        output = self.linear_final(context)

        output = self.layer_norm(residual + output)

        return output, attention

In [0]:
class ScaledDotProductAttention(torch.nn.Module):
    def __init__(self, attention_dropout = 0.0):
        super(ScaledDotProductAttention, self).__init__()
        self.dropout = torch.nn.Dropout(attention_dropout)
        self.softmax = torch.nn.Softmax(dim = 2)
    def forward(self, q, k, v, scale=None, Mask=None):

        """
        :param q:查询向量，（Batch_size, Sentence_Length, model_dim/headed_count）
        :param k: 键向量，（Batch_size, Sentence_Length, model_dim/headed_count）
        :param v: 值向量，（Batch_size, Sentence_Length, model_dim/headed_count）
        :param scale: 缩放因子，浮点标量
        :param Mask: Mask矩阵，对Attention矩阵进行Mask,与Attention矩阵的维度相同
        :return: Context，上下文张量；Attention,Attention张量
        """
        Attention = torch.bmm(q, k.transpose(1, 2))
        if scale:
            Attention *= scale
        # if Mask:
        #print(Attention.size(), Mask.size())
        Attention = Attention.masked_fill(Mask, -np.inf)

        Attention = self.softmax(Attention)
        Attention = self.dropout(Attention)
        context = torch.bmm(Attention, v)

        return context, Attention

In [0]:
class embedding_layer(torch.nn.Module):
    def __init__(self, vocab_size, model_dim):
        super(embedding_layer, self).__init__()
        self.vocab_size = vocab_size
        self.model_dim = model_dim
        self.Embedding_Matrix = torch.nn.Embedding(vocab_size, model_dim)
    def forward(self, X):
        return self.Embedding_Matrix(X) * (self.model_dim ** (0.5))
    def share_weight_linear(self, X):
        """
        :param X:Decoder输出，（batch_size, sentence_length, model_dim）
        :return: (batch_size, sentence_length, vocab_size)
        """
        batch_size = X.size(0)
        sentence_length = X.size(1)
        model_dim = X.size(2)
        out = X.view(-1, model_dim)
        out = torch.mm(out, self.Embedding_Matrix.weight.permute(1, 0))
        out = out.view(batch_size, sentence_length, self.vocab_size)
        return  out


In [0]:
class PositionEmbedding(torch.nn.Module):
    def __init__(self, Max_Sentence_Length, Model_dim):
        super(PositionEmbedding, self).__init__()

        #生成位置嵌入矩阵
        position_encoding = np.array([[pos / np.power(10000, 2.0 * (i // 2)/Model_dim) for i in range(Model_dim)]for pos in range(Max_Sentence_Length)])
        position_encoding[:, 0::2] = np.sin(position_encoding[:, 0::2])
        position_encoding[:, 1::2] = np.cos(position_encoding[:, 1::2])
        position_encoding = torch.Tensor(position_encoding)

        pad_row =  torch.zeros((1, Model_dim))

        position_encoding = torch.cat((pad_row, position_encoding), dim=0)

        self.embedding = torch.nn.Embedding.from_pretrained(position_encoding, freeze=True)

    def forward(self, input_len):
        """
        :param input:一个batch的句子长度张量，（batch_size, 1）
        :return: 一个batch句子的位置嵌入矩阵，（batch_size, max(input_len), Model_dim）
        """

        max_len = torch.max(input_len)
        position = torch.LongTensor([list(range(1, k+1)) + [0] * (max_len - k).item() for k in input_len]).cuda()
        out = self.embedding(position)
        return out

In [0]:
def padding_mask(seq_k, seq_q):
    len_q = seq_q.size(1)
    pad_mask = seq_k.eq(0)
    pad_mask = pad_mask.unsqueeze(1).expand(-1, len_q, -1)  # shape [B, L_q, L_k]
    return pad_mask
def sequence_mask(seq):
    batch_size, seq_len = seq.size()
    mask = torch.triu(torch.ones((seq_len, seq_len), dtype=torch.uint8),
                    diagonal=1)
    mask = mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]
    return mask
class EncoderLayer(torch.nn.Module):
    def __init__(self, model_dim=512, headed_count=8, ffn_dim=2048, dropout=0.0):
        super(EncoderLayer, self).__init__()
        self.MultiHeadedAttention = MultiHeadedAttention(model_dim, headed_count, dropout)
        self.FeedForward = FeedForward(model_dim, ffn_dim)
    def forward(self, X, Mask=None):
        out, attention = self.MultiHeadedAttention(X, X, X, Mask)
        out = self.FeedForward(out)
        return out, attention
class Encoder(torch.nn.Module):
    def __init__(self, source_vocab_size, Max_Sentence_Length, num_layers=6, model_dim=512, headed_count=8, ffn_dim=2048, dropout=0.0):
        super(Encoder, self).__init__()
        self.encoder = torch.nn.ModuleList(
            [EncoderLayer(model_dim, headed_count, ffn_dim, dropout)  for _ in range(num_layers)]
        )

        self.position_embedding = PositionEmbedding(Max_Sentence_Length=Max_Sentence_Length, Model_dim=model_dim)
        self.word_embedding = torch.nn.Embedding(source_vocab_size, model_dim)#Word_Embedding(pretrained)

    def forward(self,X, input_len, padding_mask):

        word_embedding = self.word_embedding(X)
        position_embedding = self.position_embedding(input_len)
        out = word_embedding + position_embedding

        attentions = []
        for encoder in self.encoder:
            out, attention = encoder(out, padding_mask)
            attentions.append(attention)

        return out, attentions

In [0]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, model_dim, head_count=8, ffn_dim=2048, dropout=0.0):
        super(DecoderLayer, self).__init__()
        self.EncoderDecoderAttention = MultiHeadedAttention(model_dim, head_count, dropout)
        self.Self_Attention = MultiHeadedAttention(model_dim, head_count, dropout)
        self.FeedForward = FeedForward(model_dim, ffn_dim, dropout)
    def forward(self, encoder_output, X=None, self_attention_mask=None, encoder_decoder_attention_mask=None):
        """
        :param X:上一层decoder的输出或者词向量； （batch_size, sentence_length, model_dim）
        :param encoder_output: encoder的输出； (batch_size, sentence_length, model_dim)
        :param mask_pad: 句子补齐产生的0向量， 负无穷填充； (batch_size, sentence_length, sentence_length)
        :param mask_sequence: attention执行过程为防止前面的词语可以看到后面的词语进行负无穷填充； (batch_size, sentence_length, sentence_length)
        :return: 一层decoder的输出，self-attention矩阵，EncoderDecoder_Attention矩阵
        """
        decoder_output, Self_attention = self.Self_Attention(X, X, X, self_attention_mask)

        decoder_output, EncoderDecoder_attention = self.EncoderDecoderAttention(decoder_output, encoder_output, encoder_output, encoder_decoder_attention_mask)

        output = self.FeedForward(decoder_output)

        return output, Self_attention, EncoderDecoder_attention
class Decoder(torch.nn.Module):
    def __init__(self, num_layer=6, model_dim=512, head_count=8, ffn_dim=2048, dropout=0.0):
        super(Decoder, self).__init__()

        self.num_layer = num_layer
        # self.word_embedding = WordEmbedding(pretrained)
        # self.Position_Embedding = PositionEmbedding(max_sentence_length, model_dim)
        self.decoder_layer = torch.nn.ModuleList([DecoderLayer(model_dim, head_count, ffn_dim, dropout) for _ in range(num_layer)])

    def forward(self, encoder_output, self_attention_mask, encoder_decoder_attention_mask, X=None):
        """
        :param X:目标语言输入； （batch_size, Sentence_Length）
        :param input_len: 目标语言句子长度，一个batch中每个句子的长度；（batch_size, 1）
        :param encoder_output: encoder的输出；（batch_size, model_dim）
        :return: Decoder输出
        注：这里的padding有点特殊，在self-attention应用padding_mask和sequence_mask的加和，在encoder-decoder attention中只应用padding_mask
        """
        output = X
        #mask = torch.gt((mask_padding+mask_context), 0)

        self_attention = []
        encoderdecoder_attention = []
        for decoder in self.decoder_layer:
            output, Self_attention, EncoderDecoder_attention = decoder(encoder_output, output, self_attention_mask, encoder_decoder_attention_mask)
            self_attention.append(Self_attention)
            encoderdecoder_attention.append(EncoderDecoder_attention)

        return output, self_attention, encoderdecoder_attention

In [0]:
class FeedForward(torch.nn.Module):
    def __init__(self, model_dim=512, ffn_dim=2048, dropout=0.0):
        super(FeedForward, self).__init__()
        self.w1 = torch.nn.Conv1d(model_dim, ffn_dim, 1)
        self.w2 = torch.nn.Conv1d(ffn_dim, model_dim, 1)
        self.dropout = torch.nn.Dropout(dropout)
        self.layer_norm = torch.nn.LayerNorm(model_dim)
    def forward(self, X):
        """
        :param X:输入， （Batch_size, Sentence_Length, model_dim）
        :return: 输出， （Batch_size, Sentence_Length, model_dim）

        """
        output = self.w2(F.relu(self.w1(X.transpose(1, 2))))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(X+output)

        return output

In [0]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [0]:
def padding_mask(seq_k, seq_q):
    len_q = seq_q.size(1)
    pad_mask = seq_k.eq(0)
    pad_mask = pad_mask.unsqueeze(1).expand(-1, len_q, -1)  # shape [B, L_q, L_k]
    return pad_mask.cuda()
def sequence_mask(seq):
    batch_size, seq_len = seq.size()
    mask = torch.triu(torch.ones((seq_len, seq_len), dtype=torch.uint8),
                    diagonal=1)
    mask = mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]
    return mask.cuda()
class transformer(torch.nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, max_sentence_length, num_layers=6, model_dim=512, headed_count=8, ffn_dim=2048, dropout=0.0):
        super(transformer, self).__init__()
        self.Encoder = Encoder(source_vocab_size, max_sentence_length, num_layers, model_dim, headed_count, ffn_dim, dropout)
        self.Decoder = Decoder(num_layers, model_dim, headed_count, ffn_dim, dropout)
        self.Position_Embedding = PositionEmbedding(max_sentence_length, model_dim)
        self.embedding_layer = embedding_layer(target_vocab_size, model_dim)
        self.linear = torch.nn.Linear(model_dim, target_vocab_size)
    def forward(self, source_word, source_len, target_word, target_len):
        #生成mask

        source_self_attention_mask = padding_mask(source_word, source_word)

        target_padding_mask = padding_mask(target_word, target_word)
        target_sequence_mask = sequence_mask(target_word)
        target_self_attention_mask = (target_padding_mask + target_sequence_mask).gt(0)
        target_encoder_decoder_mask = padding_mask(source_word, target_word)

        #生成嵌入
        #source_word_embedding = self.embedding_layer(source_word)
        #source_position_embedding = self.Position_Embedding(source_len)
        #source_embedding = source_word_embedding + source_position_embedding
        target_word_embedding = self.embedding_layer(target_word)
        target_position_embedding = self.Position_Embedding(target_len)
        target_embedding = target_word_embedding + target_position_embedding

        #encoder
        encoder_output, encoder_attention = self.Encoder(source_word, source_len, source_self_attention_mask)

        #decoder
        decoder_output, decoder_self_attention, decoder_encoder_decoder_attention = self.Decoder(encoder_output, target_self_attention_mask, target_encoder_decoder_mask, target_embedding)

        #生成logits
        logits = self.embedding_layer.share_weight_linear(decoder_output)
        # logits = self.linear(decoder_output)
        # del source_self_attention_mask, target_self_attention_mask, target_encoder_decoder_mask
        # del target_sequence_mask, target_padding_mask, source_embedding
        # del source_position_embedding, target_embedding, target_position_embedding, encoder_output

        return logits, encoder_attention, decoder_self_attention, decoder_encoder_decoder_attention

In [0]:
def loss(logits, target, smoothing, vocabsize):
    t1 = target.size(0)
    t2 = target.size(1)
    confidence = 1 - smoothing
    low_confidence = (1 - confidence)/(vocabsize - 1)
    gold = torch.zeros_like(logits).scatter(2, target.unsqueeze(-1), 1)
    gold = confidence * gold + low_confidence * (1 - gold)
    logits_softmax = F.log_softmax(logits, dim=-1)

    non_pad_mask = target.ne(Constants.PAD)
    count = non_pad_mask.sum(dim=-1).sum(dim=-1)
    loss = -(logits_softmax * gold).sum(dim=-1)
    loss = loss.masked_select(non_pad_mask).sum()

    return loss/count
def get_bleu(logits, target, index2word):
    pre = torch.argmax(logits, dim=2)
    all_bleu = []
    for i in range(logits.size(0)):
        candidate = [index2word[str(k.item())] for k in pre[i]]
        reference = [index2word[str(k.item())] for k in target[i] if k.item()!= Constants.PAD]
        if Constants.EOS_WORD in candidate:c_end = candidate.index(Constants.EOS_WORD)
        else:continue
        can = candidate[:c_end+1]

        if Constants.EOS_WORD in reference:r_end = reference.index(Constants.EOS_WORD)
        else:continue
        ref = reference[:r_end+1]
        bleu = sentence_bleu([ref], can)
        print(ref, can)
        all_bleu.append(bleu)
    return np.average(all_bleu)

In [0]:
def read_from_file(path, max_sentence_length, keep_case):
    All = []
    trimed_sentence_count = 0
    with open(path, encoding="utf-8") as file:
        for i, data in enumerate(file):
            if not keep_case:
                data = str(data).strip().lower()
            words = data.strip()
            if(len(words)>max_sentence_length):
                trimed_sentence_count += 1
            words_trimed = words[:max_sentence_length]
            if words_trimed:
                All += [[Constants.BOS] + words_trimed + [Constants.EOS]]
            else:
                All += [None]
    return All
def build_word2index(path1, path2):
    word2index = {
        Constants.BOS_WORD: Constants.BOS,
        Constants.EOS_WORD: Constants.EOS,
        Constants.PAD_WORD: Constants.PAD,
        Constants.UNK_WORD: Constants.UNK}
    with open(path1, encoding="utf-8") as file:
        for i, data in enumerate(file):
            if (word2index.get(str(data).strip())==None):
                word2index[str(data).strip()] = len(word2index)
            else:
                continue
    with open(path2, encoding="utf-8") as file:
        for i, data in enumerate(file):
            if (word2index.get(str(data).strip())==None):
                word2index[str(data).strip()] = len(word2index)
            else:
                continue
    with open("./data/word2index.json", "w", encoding="utf-8") as file:
        w2i = json.dumps(word2index)
        file.write(w2i)
    return  word2index
def convert_word_2_index(sentence, word2index):
    return [word2index.get(k, Constants.UNK) for k in sentence]
class subDataset(Dataset.Dataset):
    def __init__(self, source, target):
        super(subDataset, self).__init__()
        self.source = open(source, "r", encoding="utf-8")
        self.target = open(target, "r", encoding="utf-8")
        self.len = 0
        a = 0
        for i,data in enumerate(self.source):
            a = i
        self.len = a+1
        self.source = open(source, "r", encoding="utf-8")
        with open("/content/drive/My Drive/data/w2i.json", encoding="utf-8") as file:
            self.w2i = [json.loads(k) for k in file][0]
        with open("/content/drive/My Drive/data/dew2i.json", encoding="utf-8") as file:
            self.dew2i = [json.loads(k) for k in file][0]
    def __len__(self):
        return self.len
    def __getitem__(self, item):
        source = convert_word_2_index(str(self.source.readline()).strip().split(), self.w2i)
        source_len = len(source)
        target = convert_word_2_index(str(self.target.readline()).strip().split(), self.dew2i)
        target_len = len(target)
        return source, source_len, target, target_len
class Dataloader():
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.len = dataset.len
    def get_batch(self):
        source = []
        source_len = []
        target_input = []
        target_len_input = []
        target_output = []
        target_len_output = []
        for i in range(self.batch_size):
            s1, sl1, t1, tl1 = self.dataset.__getitem__(1)
            source.append(s1)
            source_len.append(sl1)
            target_input.append(t1[:-1])
            target_len_input.append(tl1-1)
            target_output.append(t1[1:])
            target_len_output.append(tl1-1)
        source_max_len = np.max(source_len)
        target_max_len_input = np.max(target_len_input)
        target_max_len_output = np.max(target_len_output)
        source = [k + list(np.zeros(source_max_len-len(k))) for k in source]
        target_input = [k + list(np.zeros(target_max_len_input-len(k))) for k in target_input]
        target_output = [k + list(np.zeros(target_max_len_output-len(k))) for k in target_output]
        source_len = [source_max_len for _ in range(len(source))]
        target_len_input = [target_max_len_input for _ in range(len(target_input))]
        target_len_output = [target_max_len_output for _ in range(len(target_output))]
        return torch.LongTensor(source).cuda(), torch.LongTensor(source_len).cuda(), torch.LongTensor(target_input).cuda(), torch.LongTensor(target_len_input).cuda(), torch.LongTensor(target_output).cuda(), torch.LongTensor(target_len_output).cuda()


In [0]:
class ARG():
  def __init__(self):
    self.batch_size = 40
    self.model_dim = 512
    self.epochs = 10
    self.head_count = 8
    self.dropout = 0.5
    self.save_model = "/content/drive/My Drive/transformer/transformer"
    self.label_smoothing = True
    self.learning_rate = 0.1
    self.max_sentence_length = 110
    self.vocab_size = 85718 + 1
    self.num_layer = 6
    self.ffn_dim = 2048
    self.warm_up_steps = 4000
    self.smoothing = 0.1
    self.source_vocab_size = 25809+1
    self.target_vocab_size = 24333+1
opt = ARG()

In [0]:
def train():

    #build_word2index("./data/vocab.50K.en", "./data/vocab.50K.de")
    with open("/content/drive/My Drive/data/dei2w.json", encoding="utf-8") as file:
        index2word = [json.loads(k) for k in file][0]

    model = transformer(opt.source_vocab_size, opt.target_vocab_size, opt.max_sentence_length, opt.num_layer, opt.model_dim, opt.head_count, opt.ffn_dim, opt.dropout)

    print('# generator parameters:', sum(param.numel() for param in model.parameters()))
    device = torch.device("cuda")
    # model = torch.nn.DataParallel(model, [0,1])
    model.to(device)
    model_state_dict = model.state_dict()
    optimizer = ScheduledOptim(
        optim.Adam(
            filter(lambda x: x.requires_grad, model.parameters()),
            betas=(0.9, 0.98), eps=1e-09),
        opt.model_dim, opt.warm_up_steps)
    checkpoint = {
        'model': model_state_dict,
        'settings': opt,
        'epoch': 0}
    torch.save(checkpoint, "/content/drive/My Drive/Transformer%d" % 0)
    # optimizer = optim.Adam(model.parameters(), lr = 0.9)
    for j in range(opt.epochs):

        dataset = subDataset("/content/drive/My Drive/data/pre/train_pre.en", "/content/drive/My Drive/data/pre/train_pre.de")
        dataloader = Dataloader(dataset, opt.batch_size)
        bar = tqdm(range(dataset.len // opt.batch_size))
        for i in bar:

            source, source_len, target_input, target_len_input, target_output, target_len_output = dataloader.get_batch()
            output, encoder_attention, decoder_self_attention, decoder_encoder_decoder_attention = model(source, source_len, target_input, target_len_input)

            torch.cuda.empty_cache()

            L = loss(output, target_output, opt.smoothing, opt.target_vocab_size)

            if(i%100 == 0):
                bleu = get_bleu(output, target_output, index2word)
                print(bleu)
            bar.set_description("loss:%f" % (L.item()))
            optimizer.zero_grad()
            L.backward()
            optimizer.step_and_update_lr()
            # if(i % 10000 == 0):
            #     temp = ""
            #     input(temp)
            #     if(temp == "go"):
            #         continue
            #     else:
            #         encoder_output = model.Encoder()
            #         model.Decoder()\

        model_state_dict = model.state_dict()
        checkpoint = {
            'model': model_state_dict,
            'settings': opt,
            'epoch': j}
        torch.save(checkpoint, "/content/drive/My Drive/Transformer%d"%j)


In [40]:
train()

# generator parameters: 82409230



  0%|          | 0/111721 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)

loss:285.284760:   0%|          | 0/111721 [00:00<?, ?it/s]

nan



loss:42.439213:   0%|          | 100/111721 [00:36<8:09:15,  3.80it/s]/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)

loss:40.631134:   0%|          | 100/111721 [00:36<8:09:15,  3.80it/s]

['<unk>', '<unk>', 'einiger', '<unk>', 'des', '<unk>', ',', '<unk>', 'statt', '<unk>', '5', '<unk>', 'tatsächlich', '45', '<unk>', '<unk>', 'beim', '<unk>', '.', '</s>'] ['<unk>', 'konto', '<unk>', ',', 'ernannte', 'rechtsvorschrift', '<unk>', ',', '<unk>', '<unk>', 'ausnahmeregelungen', '</s>']
['<unk>', '<unk>', 'und', 'stilvolles', '<unk>', 'mit', '<unk>', '<unk>', '(', '<unk>', 'in', '/', 'im', '<unk>', 'und', '<unk>', ')', 'und', 'gemütlicher', '<unk>', 'im', '<unk>', '.', '</s>'] ['<unk>', 'wortmeldung', 'zentraler', 'und', 'aps', '<unk>', 'kommission', '<unk>', '<unk>', '<unk>', '<unk>', 'briten', '<unk>', 'ausgeräumt', 'entgegengesetzte', 'und', 'syntax', 'nelson', 'und', 'hybrid', '<unk>', 'spaß', 's', 'volksabstimmungen', 'wertvolles', '<unk>', '<unk>', 'even', 'stadium', 'ixquick', 'kultur', 'faro', 'einreichen', 'sinnvoll', 'tauchen', '<unk>', 'begleichen', 'takt', '1920', 'finanzrahmens', '</s>']
['<unk>', '<unk>', '<unk>', 'im', '<unk>', 'war', 'der', '<unk>', 'teuer', '.

...

['<unk>', '<unk>', 'de', '<unk>', ',', 'betrachtet', 'als', 'eine', 'der', 'besten', 'der', '<unk>', ',', 'ist', 'nur', '100', 'm', 'von', 'der', '<unk>', 'entfernt', '.', '</s>'] ['<unk>', ',', ',', '<unk>', ',', '<unk>', ',', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', ',', '<unk>', ',', '<unk>', '<unk>', ',', '<unk>', '<unk>', ',', '.', '</s>']
['<unk>', 'können', '<unk>', 'viele', 'französischen', '<unk>', 'und', '<unk>', 'in', 'der', '<unk>', 'finden', '.', '</s>'] ['<unk>', ',', ',', ',', '<unk>', '<unk>', ',', '<unk>', '.', '<unk>', '<unk>', ',', '<unk>', '</s>']
['<unk>', 'de', 'la', '<unk>', 'und', '<unk>', 'de', '<unk>', ',', 'beide', 'nah', 'beim', '<unk>', ',', 'sind', 'berühmt', 'für', 'ihr', '<unk>', '<unk>', '.', '</s>'] ['<unk>', ',', '<unk>', '<unk>', ',', '<unk>', ',', '<unk>', ',', '<unk>', '<unk>', ',', '<unk>', ',', '<unk>', ',', ',', '<unk>', '<unk>', ',', '.', '</s>']
['<unk>', ',', 'da', 'die', '<unk>', 'im', '6.', '<unk>', 'ist', ',', 'wird', '<unk>', 'der', '


loss:5.816508:   2%|▏         | 1700/111721 [10:03<15:24:32,  1.98it/s]

['<unk>', 'siehe', ',', 'auch', 'die', '<unk>', '<unk>', ',', 'die', 'die', '<unk>', 'und', 'die', '<unk>', 'unserer', '<unk>', 'von', 'der', '<unk>', 'an', ',', 'da', 'sie', '<unk>', 'verlassen', 'haben', ',', 'bis', 'jetzt', 'enthalten', ',', 'und', 'sie', 'sind', 'wahr', ';', 'und', 'wir', 'können', 'von', 'ihrer', '<unk>', 'wissen', ',', 'weil', 'wir', 'sie', 'vor', '<unk>', 'haben', '.', '</s>'] ['<unk>', '<unk>', '<unk>', 'daß', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '.', '<unk>', 'zu', '<unk>', '<unk>', 'daß', '<unk>', '<unk>', '<unk>', '<unk>', 'und', '<unk>', '<unk>', '.', '<unk>', '<unk>', '<unk>', '.', 'und', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '.', '</s>']
['7', '<unk>', 'nun', ',', 'meine', '<unk>', ',', 'möchte', 'ich', ',', 'daß', 'ihr', 'daran', 'denkt', ',', 'eifrig', 'darin', 'zu', 'a', 'forschen', ',',


loss:6.271372:   2%|▏         | 1771/111721 [10:28<10:13:31,  2.99it/s]

KeyboardInterrupt: ignored

In [0]:
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
import json
import torch.utils.data.dataset as Dataset
import torch.optim as optim

In [0]:
a = "where is shanghai?"
b = "where is beijing?"

In [0]:
sentence_bleu([a], b)